In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/pairsim_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-08-24 12:42:41.980920


In [5]:
def grouper(n, iterable):
    """
    >>> list(grouper(3, 'ABCDEFG'))
    [['A', 'B', 'C'], ['D', 'E', 'F'], ['G']]
    """
    iterable = iter(iterable)
    return iter(lambda: list(itertools.islice(iterable, n)), [])


import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]

cols = ["patent", "gyear", "inv_msa", "naics_name", "primclass"]
tp_cols = ["tp", "tp_gyear", "tp_inv_msa", "tp_naics_name", "tp_primclass"]
op_cols = ["op", "op_gyear", "op_inv_msa", "op_naics_name", "op_primclass"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

get_pair_sim = False
get_field_sim = False
get_tp_field_sim = True

if get_pair_sim == True:
    try:
        for k in ["naics_name", "primclass"]:
#         for k in ["primclass"]:
            # 1. Load data
            f = "local_{0}_samp_0723.parq".format(k)

            print("computing dataframe")
            print(datetime.datetime.now())
            l2 = dd.read_parquet(pathdir+f, columns=["tp", "op"]).compute()
            l2 = l2.drop_duplicates()
            print(len(l2))
            print("Finished")
            print(datetime.datetime.now())
        
            o_f = "{0}_sim_0726.parq".format(k)
                
            print("Getting row values")
            print(datetime.datetime.now())
            pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
            pat_dict = dict(zip(pat_dict, range(len(pat_dict))))

            for dm in dms:
                print((k,dm,"started"))
                print("Loading matrix and dict")
                print(datetime.datetime.now())
                # Store copy as array
                l3 = l2.loc[l2["tp"].isin(pat_dict.keys()) & l2["op"].isin(pat_dict.keys()), ["tp", "op"]].copy()

                if dm == "ldavecs":
                    ncols = 60
                else:
                    ncols = 100

                pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
        .to_pandas().values

                # Convert to chunks
                print("Getting chunks")
                print(datetime.datetime.now())
                # Split into chunks
                n_rows = 3000
                n_chunks = int(np.round(len(l3)/n_rows))
                tp_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["tp"].iteritems()]])
                op_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["op"].iteritems()]])
                del(pm)
                chunks = itertools.zip_longest(tp_chunks, op_chunks)

                print("Getting patent pair cosine similarity")
                print(datetime.datetime.now())
                # Cosine

                cos_dis = np.empty(len(l3))

                for r, c in enumerate(chunks):
                    cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

                l3["sim_{0}".format(dm)] = 1-cos_dis
                l2 = l2.merge(l3, how = "left", on = ["tp", "op"])
                del(l3)
                print("finished")
                print(datetime.datetime.now())
            fastparquet.write(pathdir+o_f, l2, compression="GZIP")
            del(o_f)
            del(l2)
    except Exception as e:
        logging.exception("message")  

if get_field_sim == True:
    try:
        for k in ["naics_name", "primclass"]:
        
            # Define samp file
            f = "local_{0}_samp_0723.parq".format(k)
            
            l2 = dd.read_parquet(pathdir+f, \
                                 columns=["tp", "op"]).compute()
            l2 = l2.drop_duplicates()
            
            # Merge with info in invpat
            ip_tp = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(cols)\
                .rename(columns=dict(zip(cols, tp_cols)))
            l2 = l2.merge(ip_tp, how ="left", on = "tp")
            op_tp = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(cols)\
                .rename(columns=dict(zip(cols, op_cols)))
            l2 = l2.merge(op_tp, how ="left", on = "op")
            del(ip_tp, op_tp)
            
            # Save
            s_f = "{0}_mf_samp_0726.pkl".format(k)
            l2.to_pickle(pathdir+s_f)
            del(l2)
            
            get_comparison = {"naics_name": ["naics_msa", "pc", "pc_msa"],
                             "primclass": ["pc_msa"]}
            
            get_dicts = {"naics_msa": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_naics_name_msa.pkl", 
                        "pc": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_primclass_all.pkl",
                        "pc_msa": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_primclass_msa.pkl"}
                
            
                
            for comp in get_comparison[k]:
                
                # Read file
                l3 = pd.read_pickle(pathdir+s_f)
                # Define dictionary and rows for each set of comparisons
                if comp == "naics_msa":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_naics_name",
                            "op_gyear", "op_inv_msa", "op_naics_name"]]
                    l3["tp_comp"] = list(zip(l3["tp_inv_msa"],l3["tp_naics_name"], l3["tp_gyear"]))
                    l3["op_comp"] = list(zip(l3["op_inv_msa"],l3["op_naics_name"], l3["tp_gyear"]))


                elif comp == "pc":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_primclass", "tp_naics_name",
                            "op_gyear", "op_inv_msa", "op_naics_name", "op_primclass"]]

                    l3["tp_comp"] = list(zip(l3["tp_primclass"], l3["tp_gyear"]))
                    l3["op_comp"] = list(zip(l3["op_primclass"], l3["tp_gyear"]))

                elif comp == "pc_msa":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_primclass", "tp_naics_name",
                            "op_inv_msa", "op_naics_name", "op_primclass"]]

                    l3["tp_comp"] = list(zip(l3["tp_inv_msa"],l3["tp_primclass"], l3["tp_gyear"]))
                    l3["op_comp"] = list(zip(l3["op_inv_msa"],l3["op_primclass"], l3["tp_gyear"]))
                else:
                    print("error this doesn't exist")
                    continue
                
                # Drop duplicates
                print(len(l3))
                l3 = l3.drop_duplicates(["tp_comp", "op_comp"], keep="first")
                print(len(l3))
                
                # Save copy of dataframe for merging
                l2 = l3.copy()
                print(l2.head())
                
                for dm in ["ldavecs", "docvecs"]:
                    
                    for av in ["med", "mean"]:
                        print((k,comp,dm))
                        print(datetime.datetime.now())
                        # Get dictionary
                        c_dict = pd.read_pickle(get_dicts[comp].format(av, dm))
                        c_dict = c_dict.T.to_dict("list") 
                       # Make sure keys are in dictionary
                        l3 = l2[["tp_comp", "op_comp"]].copy()
                        l3 = l3.loc[l3["tp_comp"].isin(c_dict.keys()) & l3["op_comp"].isin(c_dict.keys())]
                        # Select the key columns 

                        # # 5. Get chunks
                        print("Get chunks")
                        print(datetime.datetime.now())
                        n_rows = 3000
                        n_chunks = int(np.round(len(l3)/n_rows))

                        %time tp_chunks = grouper(n_rows, [c_dict[i[1]] for i in l3["tp_comp"].iteritems()])
                        %time op_chunks = grouper(n_rows, [c_dict[i[1]] for i in l3["op_comp"].iteritems()])

                        # 3. Grouped chunks
                        chunks = itertools.zip_longest(tp_chunks, op_chunks)
                        # Delete dictionary
                        del(c_dict)

                        # 4. Get similarities
                        print("Getting average similarity")
                        print(datetime.datetime.now())
                        # Cosine

                        cos_dis = np.empty(len(l3))

                        for r, c in enumerate(chunks):
                            cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

                        l3["sim_{0}_{1}_{2}".format(av, dm, comp)] = 1-cos_dis
                        print(l3.head())

                        del(cos_dis)

                        l2 = l2.merge(l3[["tp_comp","op_comp", "sim_{0}_{1}_{2}".format(av, dm,comp)]],\
                                      how = "left", on = ["tp_comp", "op_comp"])
                        del(l3)
                        print((k,dm, "finished"))
                        print(datetime.datetime.now())
                        print(l2.sample(5))
                        print(len(l2))

#                 l2 = l2.drop(["tp_comp", "op_comp"],1)
                l2 = l2.drop_duplicates()
                # Define out file
                o_f = "{0}_{1}_0726.pkl".format(k,comp)
                l2.to_pickle(pathdir+o_f)
                del(l2)
            
    except Exception as e:
        logging.exception("message")
        
if get_tp_field_sim == True:
    try:
        for k in ["naics_name"]:
            # Already defined
        
#             # Define samp file
#             f = "local_{0}_samp_0723.parq".format(k)
            
#             l2 = dd.read_parquet(pathdir+f, \
#                                  columns=["tp", "op"]).compute()
#             l2 = l2.drop_duplicates()
            
#             # Merge with info in invpat
#             ip_tp = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(cols)\
#                 .rename(columns=dict(zip(cols, tp_cols)))
#             l2 = l2.merge(ip_tp, how ="left", on = "tp")
#             op_tp = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(cols)\
#                 .rename(columns=dict(zip(cols, op_cols)))
#             l2 = l2.merge(op_tp, how ="left", on = "op")
#             del(ip_tp, op_tp)
            
#             # Save
#             s_f = "{0}_mf_samp_0726.pkl".format(k)
#             l2.to_pickle(pathdir+s_f)
#             del(l2)
            
            # Not looking at MSA NAICS
            get_comparison = {"naics_name": ["pc", "pc_msa"],
                             "primclass": ["pc_msa"]}
            
            get_dicts = {"naics_msa": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_naics_name_msa.pkl", 
                        "pc": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_primclass_all.pkl",
                        "pc_msa": "DataStore/2018-07-P2/MeanVecDicts0723/{0}_{1}_primclass_msa.pkl"}
                
            # Reload sample file
            s_f = "{0}_mf_samp_0726.pkl".format(k)    
            for comp in get_comparison[k]:
                
                # Read file
                l3 = pd.read_pickle(pathdir+s_f)
                # Define dictionary and rows for each set of comparisons
                if comp == "naics_msa":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_naics_name",
                            "op_gyear", "op_inv_msa", "op_naics_name"]]
                    l3["tp_comp"] = list(zip(l3["tp_inv_msa"],l3["tp_naics_name"], l3["tp_gyear"]))
                    l3["op_comp"] = list(zip(l3["op_inv_msa"],l3["op_naics_name"], l3["tp_gyear"]))


                elif comp == "pc":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_primclass", "tp_naics_name",
                            "op_gyear", "op_inv_msa", "op_naics_name", "op_primclass"]]

                    l3["tp_comp"] = list(zip(l3["tp_primclass"], l3["tp_gyear"]))

                elif comp == "pc_msa":
                    l3 = l3[["tp", "tp_gyear", "tp_inv_msa", "tp_primclass", "tp_naics_name",
                            "op_inv_msa", "op_naics_name", "op_primclass"]]

                    l3["tp_comp"] = list(zip(l3["tp_inv_msa"],l3["tp_primclass"], l3["tp_gyear"]))
                else:
                    print("error this doesn't exist")
                    continue
                
                # Save copy of dataframe for merging
                l2 = l3.copy()
                print(l2.head())
                
                # Load patent dictionary
                pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
                pat_dict = dict(zip(pat_dict, range(len(pat_dict))))
                
                for dm in ["ldavecs", "docvecs"]:
                    # Define patent matrix
                    pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
        .to_pandas().values
                    
                    for av in ["med", "mean"]:
                        print((k,comp,dm))
                        print(datetime.datetime.now())
                        # Get dictionary
                        c_dict = pd.read_pickle(get_dicts[comp].format(av, dm))
                        c_dict = c_dict.T.to_dict("list") 
                       # Make sure keys are in dictionary
                        l3 = l2[["tp", "tp_comp"]].drop_duplicates().copy()
                        l3 = l3.loc[l3["tp_comp"].isin(c_dict.keys()) & l3["tp"].isin(pat_dict.keys())]
                        # Select the key columns 

                        # # 5. Get chunks
                        print("Get chunks")
                        print(datetime.datetime.now())
                        n_rows = 3000
                        n_chunks = int(np.round(len(l3)/n_rows))
                        # Target patent
                        %time tp_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["tp"].iteritems()]])
                        # Target comparison
                        %time op_chunks = grouper(n_rows, [c_dict[i[1]] for i in l3["tp_comp"].iteritems()])

                        # 3. Grouped chunks
                        chunks = itertools.zip_longest(tp_chunks, op_chunks)
                        # Delete dictionary
                        del(c_dict)

                        # 4. Get similarities
                        print("Getting average similarity")
                        print(datetime.datetime.now())
                        # Cosine

                        cos_dis = np.empty(len(l3))

                        for r, c in enumerate(chunks):
                            cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

                        l3["sim_{0}_{1}_tp_{2}".format(av, dm, comp)] = 1-cos_dis
                        print(l3.head())

                        del(cos_dis)

                        l2 = l2.merge(l3[["tp","tp_comp", "sim_{0}_{1}_tp_{2}".format(av, dm, comp)]],\
                                      how = "left", on = ["tp", "tp_comp"])
                        del(l3)
                        print((k,dm, "finished"))
                        print(datetime.datetime.now())
                        print(l2.sample(5))
                        print(len(l2))

#                 l2 = l2.drop(["tp_comp", "op_comp"],1)
                l2 = l2.drop_duplicates()
                # Define out file
                o_f = "{0}_tp_{1}_0824.pkl".format(k,comp)
                l2.to_pickle(pathdir+o_f)
                del(l2)
            
    except Exception as e:
        logging.exception("message")


INFO:root:        tp  tp_gyear                      tp_inv_msa  tp_primclass  \
0  3987987      1976            Baltimore-Towson, MD         244.0   
1  3987987      1976            Baltimore-Towson, MD         244.0   
2  3933323      1976  Boston-Cambridge-Quincy, MA-NH         244.0   
3  4129268      1978  Boston-Cambridge-Quincy, MA-NH         244.0   
4  3957229      1976  Boston-Cambridge-Quincy, MA-NH         244.0   

                 tp_naics_name  op_gyear                      op_inv_msa  \
0  Aerospace Product and Parts      1979            Baltimore-Towson, MD   
1  Aerospace Product and Parts      1979            Baltimore-Towson, MD   
2  Aerospace Product and Parts      1978  Boston-Cambridge-Quincy, MA-NH   
3  Aerospace Product and Parts      1979  Boston-Cambridge-Quincy, MA-NH   
4  Aerospace Product and Parts      1979  Boston-Cambridge-Quincy, MA-NH   

                 op_naics_name  op_primclass        tp_comp  
0  Aerospace Product and Parts         244.0  (244

CPU times: user 591 ms, sys: 42.7 ms, total: 634 ms
Wall time: 601 ms
CPU times: user 154 ms, sys: 1.37 ms, total: 156 ms
Wall time: 148 ms


INFO:root:          tp        tp_comp  sim_med_ldavecs_tp_pc
200  4208024  (244.0, 1980)               0.115068
201  4193570  (244.0, 1980)               0.412835
203  4386750  (244.0, 1983)               0.099832
204  4204657  (244.0, 1980)               0.252747
206  4369937  (244.0, 1983)               0.259493
INFO:root:('naics_name', 'ldavecs', 'finished')
INFO:root:2018-08-24 12:55:03.904104
INFO:root:              tp  tp_gyear                               tp_inv_msa  \
4752326  6549995      2003          Bridgeport-Stamford-Norwalk, CT   
3376349  4067235      1978       San Jose-Sunnyvale-Santa Clara, CA   
963235   6865852      2005  Miami-Fort Lauderdale-Pompano Beach, FL   
2847331  5133875      1992      Tampa-St. Petersburg-Clearwater, FL   
3797037  5098901      1992              Albany-Schenectady-Troy, NY   

         tp_primclass                                      tp_naics_name  \
4752326         711.0     Semiconductors and Other Electronic Components   
3376349   

CPU times: user 583 ms, sys: 46.7 ms, total: 630 ms
Wall time: 615 ms
CPU times: user 154 ms, sys: 3.17 ms, total: 157 ms
Wall time: 153 ms


INFO:root:          tp        tp_comp  sim_mean_ldavecs_tp_pc
200  4208024  (244.0, 1980)                0.281765
201  4193570  (244.0, 1980)                0.624856
203  4386750  (244.0, 1983)                0.386520
204  4204657  (244.0, 1980)                0.483266
206  4369937  (244.0, 1983)                0.490782
INFO:root:('naics_name', 'ldavecs', 'finished')
INFO:root:2018-08-24 12:55:51.778698
INFO:root:              tp  tp_gyear                                         tp_inv_msa  \
321397   8513518      2013                                    Albuquerque, NM   
277801   4524250      1985           Hartford-West Hartford-East Hartford, CT   
4796244  6373753      2002                 San Jose-Sunnyvale-Santa Clara, CA   
2994524  6009691      2000  New York-Northern New Jersey-Long Island, NY-N...   
2472645  6408720      2002                    New Orleans-Metairie-Kenner, LA   

         tp_primclass                                     tp_naics_name  \
321397          136.0

...

INFO:root:          tp                                        tp_comp  \
200  4208024                   (Ann Arbor, MI, 244.0, 1980)   
201  4193570            (Baltimore-Towson, MD, 244.0, 1980)   
203  4386750            (Baltimore-Towson, MD, 244.0, 1983)   
204  4204657            (Baltimore-Towson, MD, 244.0, 1980)   
206  4369937  (Boston-Cambridge-Quincy, MA-NH, 244.0, 1983)   

     sim_med_docvecs_tp_pc_msa  
200                   0.145154  
201                   0.224499  
203                   0.309498  
204                   0.149684  
206                   0.120914  
INFO:root:('naics_name', 'docvecs', 'finished')
INFO:root:2018-08-24 13:06:55.927589
INFO:root:              tp  tp_gyear                                         tp_inv_msa  \
286549   4952780      1990  New York-Northern New Jersey-Long Island, NY-N...   
4508938  6922858      2005  New York-Northern New Jersey-Long Island, NY-N...   
1273154  7879759      2011                               Baltimore-Towson, 

CPU times: user 764 ms, sys: 55.2 ms, total: 819 ms
Wall time: 819 ms


INFO:root:Getting average similarity
INFO:root:2018-08-24 13:08:52.751492


CPU times: user 353 ms, sys: 1.88 ms, total: 355 ms
Wall time: 355 ms


INFO:root:          tp                                        tp_comp  \
200  4208024                   (Ann Arbor, MI, 244.0, 1980)   
201  4193570            (Baltimore-Towson, MD, 244.0, 1980)   
203  4386750            (Baltimore-Towson, MD, 244.0, 1983)   
204  4204657            (Baltimore-Towson, MD, 244.0, 1980)   
206  4369937  (Boston-Cambridge-Quincy, MA-NH, 244.0, 1983)   

     sim_mean_docvecs_tp_pc_msa  
200                    0.145154  
201                    0.236092  
203                    0.324953  
204                    0.153623  
206                    0.124449  
INFO:root:('naics_name', 'docvecs', 'finished')
INFO:root:2018-08-24 13:09:57.596374
INFO:root:              tp  tp_gyear                            tp_inv_msa  \
3332909  6179363      2001            Detroit-Warren-Livonia, MI   
1617427  4590482      1986                  Boise City-Nampa, ID   
3864081  6316458      2001  Los Angeles-Long Beach-Santa Ana, CA   
1099460  8167247      2012  Los Angeles-